### Bronze Layer For Airline PNR Data

In [2]:
%pip install great_expectations

StatementMeta(, , , Waiting, , Waiting)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.0 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 122.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 19.7 MB/s eta 0:00:00


  Attempting uninstall: posthog
    Found existing installation: posthog 3.0.2
    Not uninstalling posthog at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-676514c5-7cd8-4c22-b542-f6b4826dbc20
    Can't uninstall 'posthog'. No files were found to uninstall.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Code For Just Streaming and Aggregation


In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, FloatType
from pyspark.sql.functions import col, from_json

# Create Spark Session
spark = SparkSession.builder \
    .appName("Kafka Customer and PNR Stream") \
    .getOrCreate()

# Define schemas for customer and PNR data
customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone_number", StringType(), True),
    StructField("passport_number", StringType(), True),
    StructField("nationality", StringType(), True)
])

pnr_schema = StructType([
    StructField("pnr_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("flight_number", StringType(), True),
    StructField("departure_airport", StringType(), True),
    StructField("arrival_airport", StringType(), True),
    StructField("departure_date", TimestampType(), True),
    StructField("ticket_price", FloatType(), True),
    StructField("seat_number", StringType(), True)
])

custable = "customer_table"
deltaTablePath1 = "Tables/" + custable

pnrtable = "pnr_table"
deltaTablePath2 = "Tables/" + pnrtable

# Read customer data from Kafka topic
customers_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "158.220.124.0:9094") \
    .option("subscribe", "customers_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), customer_schema).alias("data")) \
    .select("data.*")

pnr_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "158.220.124.0:9094") \
    .option("subscribe", "pnr_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), pnr_schema).alias("data")) \
    .select("data.*")

ge_df = SparkDFDataset(customers_df)
# Example 1: Check that 'customer_id' column is unique
ge_df.expect_column_values_to_be_unique("customer_id")

# Write the customer DataFrame to a Delta table
query1 = customers_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath1).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath1 + "/checkpoint").start()
query2 = pnr_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath2).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath2 + "/checkpoint").start()

# Create the aggregated table if it doesn't exist
spark.sql("""
CREATE TABLE IF NOT EXISTS aggregated_table (
    unique_id STRING,
    customer_id_c STRING,
    customer_id_p STRING,
    first_name STRING,
    last_name STRING,
    pnr_id STRING,
    flight_number STRING,
    ticket_price FLOAT
) USING DELTA
""")

while True:
    # Query your data
    df = spark.sql("""
        SELECT c.customer_id AS customer_id_c, 
               p.customer_id AS customer_id_p, 
               c.first_name, 
               c.last_name, 
               p.pnr_id,
               p.flight_number,
               p.ticket_price
        FROM customer_table c 
        INNER JOIN pnr_table p ON c.customer_id = p.customer_id
    """)

    # Create a unique identifier for each record
    df_with_unique_id = df.withColumn("unique_id", F.expr("concat(customer_id_c, pnr_id)"))

    # Deduplicate the source data to ensure only one row per unique_id
    df_with_unique_id_deduped = df_with_unique_id.dropDuplicates(["unique_id"])

    # Perform the merge operation
    df_with_unique_id_deduped.createOrReplaceTempView("new_data")

    merge_query = """
    MERGE INTO aggregated_table AS target
    USING new_data AS source
    ON target.unique_id = source.unique_id
    WHEN MATCHED THEN
        UPDATE SET 
            target.first_name = source.first_name,
            target.last_name = source.last_name,
            target.pnr_id = source.pnr_id,
            target.flight_number = source.flight_number,
            target.ticket_price = source.ticket_price
    WHEN NOT MATCHED THEN
        INSERT (unique_id, customer_id_c, customer_id_p, first_name, last_name, pnr_id, flight_number, ticket_price)
        VALUES (source.unique_id, source.customer_id_c, source.customer_id_p, source.first_name, source.last_name, source.pnr_id, source.flight_number, source.ticket_price)
    """

    # Execute the merge query
    spark.sql(merge_query)

    time.sleep(30)  # Wait for 30 seconds before running the query again

query1.awaitTermination()
query2.awaitTermination()

#query1.awaitTermination()
#query2.awaitTermination()

StatementMeta(, , , Waiting, , Waiting)

## Code For Printing Kafka Data On Console

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Kafka Integration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()


# Kafka configuration
kafka_bootstrap_servers = "158.220.124.0:9094"  # Replace with your Kafka broker address
kafka_topic = "customers_data"                     # Replace with your Kafka topic  test_topic

# Read from Kafka
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "latest") \
  .option("failOnDataLoss", "false")\
  .load()

# Convert Kafka message key and value from binary to string
kafka_df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Define a function to print each consumed message
def process_batch(df, batch_id):
    # Print the batch id and each row
    print(f"Processing batch {batch_id}:")
    for row in df.collect():
        print(f"Key: {row['key']}, Value: {row['value']}")

# Output each message and write to console
query = kafka_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", "Files/RealTime/checkpoint") \
    .start()

query.awaitTermination()

StatementMeta(, , , Waiting, , Waiting)

## Complete Code For Our Pipeline (Run only this Cell for Demo)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, FloatType
from pyspark.sql.functions import col, from_json, window, current_timestamp
from pyspark.sql.functions import upper, regexp_replace, when, length
import great_expectations as ge
import os
import json
import pyspark.sql.functions as F


# Create Spark Session
spark = SparkSession.builder \
    .appName("Kafka Customer and PNR Stream") \
    .getOrCreate()

# Define schemas for customer and PNR data
customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone_number", StringType(), True),
    StructField("passport_number", StringType(), True),
    StructField("nationality", StringType(), True)
])

pnr_schema = StructType([
    StructField("pnr_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("flight_number", StringType(), True),
    StructField("departure_airport", StringType(), True),
    StructField("arrival_airport", StringType(), True),
    StructField("departure_date", TimestampType(), True),
    StructField("ticket_price", FloatType(), True),
    StructField("seat_number", StringType(), True)
])

custable = "Uncleaned_Customer_Table"
deltaTablePath1 = "Tables/" + custable

cleancustable = "Cleaned_Customer_Table"
deltaTablePath2 = "Tables/" + cleancustable

pnrtable = "Uncleaned_Pnr_Table"
deltaTablePath3 = "Tables/" + pnrtable

cleanpnrtable = "Cleaned_Pnr_Table"
deltaTablePath4 = "Tables/" + cleanpnrtable

# Read customer data from Kafka topic
customers_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "158.220.124.0:9094") \
    .option("subscribe", "customers_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), customer_schema).alias("data")) \
    .select("data.*")

pnr_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "158.220.124.0:9094") \
    .option("subscribe", "pnr_data") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), pnr_schema).alias("data")) \
    .select("data.*")

cleaned_customer_df = customers_df \
    .withColumn("customer_id", regexp_replace(col("customer_id"), "[^A-Za-z0-9]", "")) \
    .withColumn("first_name", regexp_replace(upper(col("first_name")), "[^A-Za-z]", "")) \
    .withColumn("last_name", regexp_replace(upper(col("last_name")), "[^A-Za-z]", "")) \
    .withColumn("email", when(col("email").rlike("^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}$"), col("email")).otherwise(None)) \
    .withColumn("phone_number", regexp_replace(col("phone_number"), "[^0-9]", "")) \
    .withColumn("phone_number", when(length(col("phone_number")) >= 10, col("phone_number")).otherwise(None)) \
    .withColumn("passport_number", regexp_replace(col("passport_number"), "[^A-Za-z0-9]", "")) \
    .withColumn("nationality", regexp_replace(upper(col("nationality")), "[^A-Za-z]", "")) 

cleaned_df = pnr_df \
    .withColumn("pnr_id", regexp_replace(col("pnr_id"), "[^A-Za-z0-9-]", "")) \
    .withColumn("customer_id", regexp_replace(col("customer_id"), "[^A-Za-z0-9]", "")) \
    .withColumn("flight_number", upper(col("flight_number"))) \
    .withColumn("flight_number", regexp_replace(col("flight_number"), "[^A-Z0-9]", "")) \
    .withColumn("departure_airport", upper(col("departure_airport"))) \
    .withColumn("arrival_airport", upper(col("arrival_airport"))) \
    .withColumn("departure_airport", when(col("departure_airport").rlike("^[A-Z]{3}$"), col("departure_airport")).otherwise(None)) \
    .withColumn("arrival_airport", when(col("arrival_airport").rlike("^[A-Z]{3}$"), col("arrival_airport")).otherwise(None)) \
    .withColumn("departure_date", when(col("departure_date").isNotNull(), col("departure_date")).otherwise(None)) \
    .withColumn("ticket_price", when(col("ticket_price") >= 0, col("ticket_price")).otherwise(None)) \
    .withColumn("seat_number", regexp_replace(col("seat_number"), "[^0-9A-Za-z]", "")) \
    .withColumn("seat_number", when(col("seat_number").rlike("^[0-9]+[A-Z]$"), col("seat_number")).otherwise(None))  # Example validation for seat number format

# Adding window for micro batches
cleaned_customer_with_timestamp_df = cleaned_customer_df \
    .withColumn("timestamp", current_timestamp())

windowed_customer_df = cleaned_customer_with_timestamp_df \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(window(col("timestamp"), "1 minute"), "customer_id") \
    .count() 

cleaned_pnr_with_timestamp_df = cleaned_df \
    .withColumn("timestamp", current_timestamp())

windowed_pnr_df = cleaned_pnr_with_timestamp_df \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(window(col("timestamp"), "1 minute"), "pnr_id") \
    .count() 


context = ge.get_context()
data_source = context.data_sources.add_spark("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": cleaned_customer_df})
# Convert Spark DataFrame to Pandas for validation with Great Expectations

output_directory = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/f2d127ee-ca13-4665-b186-31be52593744/Files/Validations_Checks/customer"  # Change this to your lakehouse path

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

output_directory2 = "abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/f2d127ee-ca13-4665-b186-31be52593744/Files/Validations_Checks/pnr"  # Change this to your lakehouse path

# Ensure the output directory exists
os.makedirs(output_directory2, exist_ok=True)

# Validating the Cleaned Data using Great Expectations
#validate_id = ge.expectations.ExpectColumnDistinctValuesToBeInSet("customer_id", cleaned_customer_df["customer_id"].unique())
#validate_firstname = ge.expectations.ExpectColumnDistinctValuesToBeInSet("first_name", cleaned_customer_df["first_name"].unique())
#validate_lastname = ge.expectations.ExpectColumnDistinctValuesToBeInSet("last_name", cleaned_customer_df["last_name"].unique())
validate_email=ge.expectations.ExpectColumnValuesToMatchRegex(column ="email",regex= r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
validate_number= ge.expectations.ExpectColumnValuesToMatchRegex(column ="phone_number",regex = r"^\+?[0-9]{10,15}$")
validate_passport=ge.expectations.ExpectColumnValuesToMatchRegex(column ="passport_number", regex =r"^[A-Z0-9]{5,9}$")
#validate_nationality=ge.expectations.ExpectColumnDistinctValuesToBeInSet("nationality", cleaned_customer_df["nationality"].unique())

# validations for PNR data
#pnridtoexist = ge.expectations.expect_column_to_exist("pnr_id")
#cusidtoexist = ge.expectations.expect_column_to_exist("customer_id")
#flightnumbertoexist = ge.expectations.expect_column_to_exist("flight_number")
#departuretoexist = ge.expectations.expect_column_to_exist("departure_airport")
#arrivaltoexist = ge.expectations.expect_column_to_exist("arrival_airport")
#departuredatetoexist = ge.expectations.expect_column_to_exist("departure_date")
#tickettoexist = ge.expectations.expect_column_to_exist("ticket_price")
#seattoexist = ge.expectations.expect_column_to_exist("seat_number")

# Validations for pnr_id, customer_id, and flight_number (non-null)
pnridnotnull = ge.expectations.ExpectColumnValuesToNotBeNull(column="pnr_id")
customeridnotnull = ge.expectations.ExpectColumnValuesToNotBeNull(column="customer_id")
flightnumbernotnull = ge.expectations.ExpectColumnValuesToNotBeNull(column="flight_number")

# Validations for departure_airport and arrival_airport (non-null and within a specific list of valid airports)
#valid_airports = ["JFK", "LAX", "ORD", "ATL", "DFW"]  # Example airport codes
#ge_df.expect_column_values_to_not_be_null("departure_airport")
#ge_df.expect_column_values_to_be_in_set("departure_airport", valid_airports)
#ge_df.expect_column_values_to_not_be_null("arrival_airport")
#ge_df.expect_column_values_to_be_in_set("arrival_airport", valid_airports)

def customer_validation_great_expectations(df, batch_id):
    validation_results = []

    # Create a Great Expectations batch
    context = ge.get_context()
    data_source = context.data_sources.add_spark("pandas")
    data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

    batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
    batch = batch_definition.get_batch(batch_parameters={"dataframe": df})
    # Validate each expectation
    validation_result_email = batch.validate(validate_email)
    validation_result_number = batch.validate(validate_number)
    validation_result_passport = batch.validate(validate_passport)

    #print(validation_result_email)
    validation_results.append({
        'batch_id': batch_id,
        'email_validation': validation_result_email['success'],
        'phone_number_validation': validation_result_number['success'],
        'passport_validation': validation_result_passport['success']
    })

    # Define the file name for the JSON file
    json_file_path = os.path.join(output_directory, f"validation_results_{batch_id}.json")

    # Write the validation results to a JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(validation_results, json_file, indent=4)

    # Print processing status
    print(f"Processed batch {batch_id} and saved results to {json_file_path}.")

def pnr_validation_great_expectation(df, batch_id):
    validation_results = []

    # Create a Great Expectations batch
    context = ge.get_context()
    data_source = context.data_sources.add_spark("pandas")
    data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

    batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
    batch = batch_definition.get_batch(batch_parameters={"dataframe": df})
    # Validate each expectation
    validation_result_pnr = batch.validate(pnridnotnull)
    validation_result_cus = batch.validate(customeridnotnull)
    validation_result_flight = batch.validate(flightnumbernotnull)
   # print(validation_result_pnr)

    validation_results.append({
        'batch_id': batch_id,
        'pnr_id_validation': validation_result_pnr['success'],
        'customer_id_validation': validation_result_cus['success'],
        'flight_number_validation': validation_result_flight['success']
    })

    # Define the file name for the JSON file
    json_file_path = os.path.join(output_directory2, f"validation_results_{batch_id}.json")

    # Write the validation results to a JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(validation_results, json_file, indent=4)

    # Print processing status
    print(f"Processed batch {batch_id} and saved results to {json_file_path}.")

# Apply the validation function to each batch of streaming data
# Validating Our Dataframes
query5 = cleaned_customer_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="60 seconds") \
    .foreachBatch(customer_validation_great_expectations) \
    .option("checkpointLocation", "Files/RealTime/checkpoint") \
    .start()
query6 = cleaned_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="60 seconds") \
    .foreachBatch(pnr_validation_great_expectation) \
    .option("checkpointLocation", "Files/RealTime/checkpoint") \
    .start()

# Write the customer DataFrame to a Delta table 
# Creating Bronze Layer For Customer and PNR
query1 = customers_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath1).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath1 + "/checkpoint").start()
query3 = pnr_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath3).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath3 + "/checkpoint").start()

# Creating Silver Layer 
query2 = cleaned_customer_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath2).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath2 + "/checkpoint").start()
query4 = cleaned_df.writeStream.outputMode("append").format("delta").option("path", deltaTablePath4).option("mergeSchema", "true").option("checkpointLocation", deltaTablePath4 + "/checkpoint").start()


#Creating Gold Layer
# Create the aggregated table if it doesn't exist
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_aggregated_table (
    unique_id STRING,
    customer_id_c STRING,
    customer_id_p STRING,
    first_name STRING,
    last_name STRING, 
    email STRING,
    phone_number STRING,
    passport_number STRING,
    nationality STRING,
    pnr_id STRING,
    flight_number STRING,
    ticket_price FLOAT,
    departure_airport STRING,
    arrival_airport STRING,
    departure_date TIMESTAMP,
    seat_number STRING

) USING DELTA
""")
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_pnr_summary (
    customer_id_c STRING,
    passenger_name STRING,
    pnr_id STRING,
    flight_number STRING,
    ticket_price FLOAT,
    seat_number STRING,
    departure_date TIMESTAMP
) USING DELTA
""")


spark.sql("""
CREATE TABLE IF NOT EXISTS gold_passenger_summary (
Passenger_Id STRING,
Passenger_name STRING,
Total_ticket_price STRING,
Average_Ticket_Price STRING,
Total_Flights STRING
) USING DELTA """)


while True:
    # Query your data
    df = spark.sql("""
        SELECT c.customer_id AS customer_id_c, 
               p.customer_id AS customer_id_p, 
               c.first_name, 
               c.last_name, 
               c.email,
               c.phone_number,
               c.passport_number,
               c.nationality,
               p.pnr_id,
               p.flight_number,
               p.ticket_price,
               p.departure_airport,
               p.arrival_airport,
               p.departure_date,
               p.seat_number
        FROM Cleaned_Customer_Table c 
        INNER JOIN Cleaned_Pnr_Table p ON c.customer_id = p.customer_id
    """)

    # Create a unique identifier for each record
    df_with_unique_id = df.withColumn("unique_id", F.expr("concat(customer_id_c, pnr_id)"))

    # Deduplicate the source data to ensure only one row per unique_id
    df_with_unique_id_deduped = df_with_unique_id.dropDuplicates(["unique_id"])

    # Perform the merge operation
    df_with_unique_id_deduped.createOrReplaceTempView("new_data")

    merge_query = """
    MERGE INTO gold_aggregated_table AS target
    USING new_data AS source
    ON target.unique_id = source.unique_id
    WHEN MATCHED THEN
        UPDATE SET 
            target.first_name = source.first_name,
            target.last_name = source.last_name,
            target.pnr_id = source.pnr_id,
            target.flight_number = source.flight_number,
            target.ticket_price = source.ticket_price,
            target.email = source.email,
            target.phone_number = source.phone_number,
            target.passport_number = source.passport_number,
            target.nationality = source.nationality,
            target.departure_airport = source.departure_airport,
            target.arrival_airport = source.arrival_airport,
            target.departure_date = source.departure_date,
            target.seat_number = source.seat_number
    WHEN NOT MATCHED THEN
        INSERT (unique_id, customer_id_c, customer_id_p, first_name, last_name, pnr_id, flight_number, ticket_price)
        VALUES (source.unique_id, source.customer_id_c, source.customer_id_p, source.first_name, source.last_name, source.pnr_id, source.flight_number, source.ticket_price)
    """

    
    df2 = spark.sql("""
        select
        customer_id_c,
        (first_name ||' '||last_name) as passenger_name,
        pnr_id,
        flight_number,
        ticket_price, 
        seat_number, 
        departure_date 
        from gold_aggregated_table;
    """)

    df2.createOrReplaceTempView("new_pnr_view")
    merge_query2 = """
    MERGE INTO gold_pnr_summary AS target
    USING new_pnr_view AS source
    ON target.pnr_id = source.pnr_id
    WHEN MATCHED THEN
        UPDATE SET 
            target.customer_id_c = source.customer_id_c,
            target.passenger_name = source.passenger_name,
            target.pnr_id = source.pnr_id,
            target.flight_number = source.flight_number,
            target.ticket_price = source.ticket_price,
            target.seat_number = source.seat_number,
            target.departure_date = source.departure_date
    WHEN NOT MATCHED THEN
        INSERT (customer_id_c, passenger_name, pnr_id, flight_number, ticket_price, seat_number, departure_date)
        VALUES (source.customer_id_c,
                source.passenger_name,
                source.pnr_id, 
                source.flight_number, 
                source.ticket_price, 
                source.seat_number, 
                source.departure_date)
    """

    df3 = spark.sql("""SELECT 
        customer_id_c ,
        (first_name ||' '||last_name) as passenger_name ,
        sum(ticket_price) as Total_ticket_price,
        avg(ticket_price) as Average_Ticket_Price,
        count(*) as Total_Flights
        FROM 
            gold_aggregated_table 
        group by customer_id_c,
        (first_name ||' '||last_name); 
        """)

    df3.createOrReplaceTempView("new_passenger")

    merge_summary_query = """
    MERGE INTO gold_passenger_summary AS target
    USING new_passenger AS source
    ON target.Passenger_Id = source.customer_id_c
    WHEN MATCHED THEN
        UPDATE SET 
            target.passenger_name = source.passenger_name,
            target.Total_ticket_price = source.Total_ticket_price,
            target.Average_Ticket_Price = source.Average_Ticket_Price,
            target.Total_Flights = source.Total_Flights
    WHEN NOT MATCHED THEN
        INSERT (Passenger_Id, passenger_name, Total_ticket_price, Average_Ticket_Price, Total_Flights)
        VALUES (source.customer_id_c,
                source.passenger_name,
                source.Total_ticket_price, 
                source.Average_Ticket_Price, 
                source.Total_Flights)
        """

    spark.sql(merge_query2)
    # Execute the merge query
    spark.sql(merge_query)
    spark.sql(merge_summary_query)

query.awaitTermination()
query1.awaitTermination()
query2.awaitTermination()
query3.awaitTermination()
query4.awaitTermination()
query5.awaitTermination()
query6.awaitTermination()

StatementMeta(, , , Waiting, , Waiting)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


Created temporary directory '/tmp/tmpr_ihim2l' for ephemeral docs site


Created temporary directory '/tmp/tmpbmxx8fyr' for ephemeral docs site


Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

...

Processed batch 582 and saved results to abfss://ed9bea8b-5f1a-4dbe-a49e-4cc0009f02db@onelake.dfs.fabric.microsoft.com/f2d127ee-ca13-4665-b186-31be52593744/Files/Validations_Checks/pnr/validation_results_582.json.


Py4JJavaError: An error occurred while calling o326.sql.
: org.apache.spark.SparkException: Job 1595 cancelled part of cancelled job group 11
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2871)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2746)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleJobGroupCancelled$4(DAGScheduler.scala:1187)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.handleJobGroupCancelled(DAGScheduler.scala:1186)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3029)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3009)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2998)
	at org.apache.spark.util.EventLoop$$anon$1.ru

StatementMeta(, , -1, Waiting, , Waiting)

StatementMeta(, , -1, Waiting, , Waiting)

StatementMeta(, , -1, Waiting, , Waiting)

## SQL Queries

In [ ]:
%%sql
--select count(*) from Cleaned_Customer_Table c  group by customer_id;
--select * from gold_aggregated_table ;
--select pnr_id, (first_name ||' '||last_name) as passenger_name,ticket_price, seat_number, departure_date from gold_aggregated_table;
-- select * from new_passenger_table where customer_id=10;
-- SELECT * from uncleaned_customer_table where customer_id=10;
-- SELECT * from cleaned_customer_table where customer_id=10;
SELECT * from uncleaned_pnr_table where customer_id=10;
SELECT * from cleaned_pnr_table where customer_id=10;
SELECT * from gold_passenger_summary where passenger_id=10;




--select * from gold_aggregated_table where customer_id_C =  '511';

StatementMeta(, , , Waiting, , Waiting)

<Spark SQL result set with 3 rows and 8 fields>

<Spark SQL result set with 3 rows and 8 fields>

<Spark SQL result set with 1 rows and 5 fields>

## Example For Great Expectations Library

In [19]:
import pandas as pd
import great_expectations as gx
from great_expectations.core.batch import Batch, BatchRequest
from great_expectations.validator.validator import Validator

# Example DataFrame (Replace with your actual DataFrame)
data = {
    "passenger_count": [1, 3, 2, 6,  5],  # 10 is an outlier (greater than max 6)
    "name": ["John", "Jane", "Doe",  "Bob", "Charlie"]
}

pandas_df = pd.DataFrame(data)

# Step 1: Initialize Great Expectations context
context = gx.get_context()
data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": pandas_df})

# Create Expectation.
expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="passenger_count", min_value=1, max_value=6
)

# Validate Batch using Expectation.
validation_result = batch.validate(expectation)

# Step 5: Validate the DataFrame with the expectation

# Step 6: Print the validation results
print(validation_result)

# Optional: Save the expectation suite or validation results


StatementMeta(, , , Waiting, , Waiting)